<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Using TensorFlow Keras, Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn.

This is your "Hello World!" of TensorFlow.

### Example TensorFlow Starter Code

```python 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(3, activation='sigmoid', input_dim=2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

results = model.fit(X,y, epochs=100)

```

### Additional Written Tasks:
1. Investigate the various [loss functions](https://www.tensorflow.org/api_docs/python/tf/keras/losses). Which is best suited for the task at hand (predicting 1 / 0) and why? 
2. What is the difference between a loss function and a metric? Why might we need both in Keras? 
3. Investigate the various [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). Stochastic Gradient Descent (`sgd`) is not the learning algorithm dejour anyone. Why is that? What do newer optimizers such as `adam` have to offer? 

1. In this case, we could either use Hinge loss or Binary cross-entropy but since the values are already 0's and 1's, the best loss function to use would be binary cross-entropy. The reason why is in the name, binary cross-entropy is a loss function specifically used for binary classification problems and is also the default loss function for any binary classification problem.

2. The loss function is what's passed into a keres model and actually optimizes the model meanwhile the metric is used to actually judge the performance of the model created. We need the loss function so we can actually optimize the model to a result we are looking for.

3. New optimizers such as adam uses a algorithm that uses very minimal memory and works well with very large datasets.

In [1]:
##### Your Code Here #####
import pandas as pd

data = { 'x1': [0,0,1,0,1,1,0],
         'x2': [0,1,0,1,0,1,0],
         'x3': [1,1,1,0,0,1,0],
         'y':  [0,1,1,0,1,0,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')
X = df[['x1', 'x2', 'x3']].values
y = df['y'].values

In [93]:
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,0
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [51]:
inputs = X.shape[1]

In [97]:
X.shape

(7, 3)

In [25]:
y.shape

(7,)

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


model = Sequential()
model.add(Dense(inputs, input_dim=inputs, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs=10)

Train on 7 samples
Epoch 1/10
7/7 [==============================] - 0s 37ms/sample - loss: 0.6835 - accuracy: 0.5714
Epoch 2/10
7/7 [==============================] - 0s 274us/sample - loss: 0.6832 - accuracy: 0.5714
Epoch 3/10
7/7 [==============================] - 0s 289us/sample - loss: 0.6829 - accuracy: 0.5714
Epoch 4/10
7/7 [==============================] - 0s 239us/sample - loss: 0.6826 - accuracy: 0.5714
Epoch 5/10
7/7 [==============================] - 0s 317us/sample - loss: 0.6823 - accuracy: 0.5714
Epoch 6/10
7/7 [==============================] - 0s 616us/sample - loss: 0.6821 - accuracy: 0.5714
Epoch 7/10
7/7 [==============================] - 0s 619us/sample - loss: 0.6818 - accuracy: 0.5714
Epoch 8/10
7/7 [==============================] - 0s 174us/sample - loss: 0.6816 - accuracy: 0.5714
Epoch 9/10
7/7 [==============================] - 0s 168us/sample - loss: 0.6813 - accuracy: 0.5714
Epoch 10/10
7/7 [==============================] - 0s 201us/sample - loss: 0.6811 

In [112]:
test = [[1,0,1]]

model.predict(test)

array([[0.46567646]], dtype=float32)

### Build a Tensor Keras Perceptron

Try to match the architecture we used on Monday - inputs nodes and one output node. Apply this architecture to the XOR-ish dataset above. 

After fitting your model answer these questions: 

Are you able to achieve the same results as a bigger architecture from the first part of the assignment? Why is this disparity the case? What properties of the XOR dataset would cause this disparity? 

Now extrapolate this behavior on a much larger dataset in terms of features. What kind of architecture decisions could we make to avoid the problems the XOR dataset presents at scale? 

*Note:* The bias term is baked in by default in the Dense layer.

In [131]:
### Compare 
model = Sequential()
model.add(Dense(3, input_dim=3, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10)

Train on 7 samples
Epoch 1/10
7/7 [==============================] - 0s 29ms/sample - loss: 0.7318 - accuracy: 0.1429
Epoch 2/10
7/7 [==============================] - 0s 230us/sample - loss: 0.7315 - accuracy: 0.1429
Epoch 3/10
7/7 [==============================] - 0s 243us/sample - loss: 0.7311 - accuracy: 0.1429
Epoch 4/10
7/7 [==============================] - 0s 215us/sample - loss: 0.7307 - accuracy: 0.1429
Epoch 5/10
7/7 [==============================] - 0s 208us/sample - loss: 0.7303 - accuracy: 0.1429
Epoch 6/10
7/7 [==============================] - 0s 230us/sample - loss: 0.7300 - accuracy: 0.1429
Epoch 7/10
7/7 [==============================] - 0s 182us/sample - loss: 0.7296 - accuracy: 0.1429
Epoch 8/10
7/7 [==============================] - 0s 241us/sample - loss: 0.7292 - accuracy: 0.1429
Epoch 9/10
7/7 [==============================] - 0s 235us/sample - loss: 0.7289 - accuracy: 0.1429
Epoch 10/10
7/7 [==============================] - 0s 236us/sample - loss: 0.7285 

In [111]:
test = [[1,0,1]]

model.predict(test)

array([[0.46567646]], dtype=float32)

## Try building/training a more complex MLP on a bigger dataset.

Use TensorFlow Keras & the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the canonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the Internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [113]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [114]:
# input image dimensions
img_rows, img_cols = 28, 28

In [115]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [116]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [117]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simplify the problem for now: Zero or all else.

In [120]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [121]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

**The hardest portion of this assignment is following the Numpy shapes in the backprop function**



In [99]:
##### Your Code Here #####

class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 784
        self.hiddenNodes = 128
        self.outputNodes = 1

        # Initial Weights
        # 3x3 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        # Categorical Cross Entropy - Would a function to do this. 
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        self.o_delta = np.expand_dims(self.o_delta, axis=1)
        self.z2_error = self.o_delta * self.weights2.T
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)[:,0]
        self.backward(X,y,o)

In [101]:
nn.o_error.shape 

(500,)

In [97]:
t = nn.o_error * nn.sigmoidPrime(nn.feed_forward(x_test[:500]))
t.shape

(500, 500)

In [100]:
nn = NeuralNetwork()

# Number of Epochs / Iterations
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print("Loss: \n", str(np.mean(np.square(y_test- nn.feed_forward(x_test)))))
    nn.train(x_train[:500],y_train[:500])

+---------EPOCH 1---------+
Loss: 
 0.098
+---------EPOCH 2---------+
Loss: 
 0.098
+---------EPOCH 3---------+
Loss: 
 0.098
+---------EPOCH 4---------+
Loss: 
 0.098
+---------EPOCH 5---------+
Loss: 
 0.098
+---------EPOCH 1000---------+
Loss: 
 0.098
+---------EPOCH 2000---------+
Loss: 
 0.098
+---------EPOCH 3000---------+
Loss: 
 0.098
+---------EPOCH 4000---------+
Loss: 
 0.098
+---------EPOCH 5000---------+
Loss: 
 0.098
+---------EPOCH 6000---------+
Loss: 
 0.098
+---------EPOCH 7000---------+
Loss: 
 0.098
+---------EPOCH 8000---------+
Loss: 
 0.098
+---------EPOCH 9000---------+
Loss: 
 0.098
+---------EPOCH 10000---------+
Loss: 
 0.098


## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?